In [47]:
import os 
import sys
import pandas as pd
import numpy as np  
from tqdm import tqdm
from typing import List
from numpy.typing import NDArray



In [48]:

 ###################TODO: change the path to the correct one#####################

# action_****mode****.txt
path_to_label = '/Users/dennisbaumann/cars_paper/data/label_split/action_train.txt'
root_data = '/Users/dennisbaumann/cars_paper/data/'

n_frames_per_seq : int = 8

In [49]:
mode = 'train'
sample_root = root_data + f'mano_{n_frames_per_seq}_{mode}/'

df = pd.read_csv(path_to_label, delimiter=' ')
print(df.head())    
ids : List[int] = df['id'].to_list()
paths : List[str] = df['path'].to_list()
action_labels : List[int] = df['action_label'].to_list()
start_acts : List[int] = df['start_act'].to_list()
end_acts : List[int] = df['end_act'].to_list()
start_frames : List[int] = df['start_frame'].to_list()
end_frames : List[int] = df['end_frame'].to_list()

print("Number of action sequences:", len(ids))

   id           path  action_label  start_act  end_act  start_frame  end_frame
0   1  subject1/h1/0            16          0       61            0        449
1   2  subject1/h1/0             8         64      127            0        449
2   3  subject1/h1/0            27        133      255            0        449
3   4  subject1/h1/0            30        317      417            0        449
4   5  subject1/h1/1            13          0       48            0        642
Number of action sequences: 569


In [25]:
os.makedirs(sample_root, exist_ok=True) # create the destination directory

sample_ids : List[NDArray] = []
# sample the frames for each action
for i in tqdm(range(len(ids))):
    start_act = start_acts[i]
    end_act = end_acts[i]
    n_frames_total = end_act-start_act+1 # total number of frames in the action sequence
    assert n_frames_total >= n_frames_per_seq, \
        f"Requested {n_frames_per_seq} samples, but action (id {ids[i]}) has only {n_frames_total} frames"

    seq_ids = np.linspace(start_acts[i], end_acts[i], n_frames_per_seq, dtype=int)
    sample_ids.append(seq_ids)

sample_ids = np.array(sample_ids)
#np.save(sample_root + 'sample_ids.npy', sample_ids)
print("Sampling IDs shape:", sample_ids.shape) # sanity check the shape of the sampled ids
print("Last sampled sequence ids:", seq_ids) # sanity check the sampling from the last action

100%|██████████| 569/569 [00:00<00:00, 7822.58it/s]

Sampling IDs shape: (569, 8)
Last sampled sequence ids: [425 434 443 452 461 470 479 489]


In [50]:
print(sample_ids)

[[  0   8  17 ...  43  52  61]
 [ 64  73  82 ... 109 118 127]
 [133 150 167 ... 220 237 255]
 ...
 [122 144 167 ... 234 257 280]
 [309 318 328 ... 356 366 376]
 [425 434 443 ... 470 479 489]]


In [52]:
manos : list[NDArray] = []
cam_pose : list[NDArray] = []
obj_pose : list[NDArray] = []
for i,id in enumerate(sample_ids):
    path = paths[i]
    manos_list_curr = []
    cam_pose_list_curr = []
    obj_pose_list_curr = []
    for j in id:

        ###################TODO: change the path to the correct one#####################
        path_curr = '/Users/dennisbaumann/cars_paper/data/' + path + f'/cam4/hand_pose_mano/{j:06d}.txt'
        cam_pose_curr = np.loadtxt('/Users/dennisbaumann/cars_paper/data/' + path + f'/cam4/cam_pose/{j:06d}.txt')
        obj_pose_curr = np.loadtxt('/Users/dennisbaumann/cars_paper/data/' + path + f'/cam4/obj_pose/{j:06d}.txt')
        
        mano_curr  = np.loadtxt(path_curr)
        cam_pose_list_curr.append(np.array(cam_pose_curr))
        manos_list_curr.append(np.array(mano_curr))
        obj_pose_list_curr.append(np.array(obj_pose_curr))
    
    manos.append(np.array(manos_list_curr))
    cam_pose.append(np.array(cam_pose_list_curr))
    obj_pose.append(np.array(obj_pose_list_curr))
    print(np.shape(manos))
    print(np.shape(cam_pose))



/Users/dennisbaumann/cars_paper/data/subject1/h1/0/cam4/hand_pose_mano/000000.txt
/Users/dennisbaumann/cars_paper/data/subject1/h1/0/cam4/hand_pose_mano/000008.txt
/Users/dennisbaumann/cars_paper/data/subject1/h1/0/cam4/hand_pose_mano/000017.txt
/Users/dennisbaumann/cars_paper/data/subject1/h1/0/cam4/hand_pose_mano/000026.txt
/Users/dennisbaumann/cars_paper/data/subject1/h1/0/cam4/hand_pose_mano/000034.txt
/Users/dennisbaumann/cars_paper/data/subject1/h1/0/cam4/hand_pose_mano/000043.txt
/Users/dennisbaumann/cars_paper/data/subject1/h1/0/cam4/hand_pose_mano/000052.txt
/Users/dennisbaumann/cars_paper/data/subject1/h1/0/cam4/hand_pose_mano/000061.txt
(1, 8, 124)
(1, 8, 16)
/Users/dennisbaumann/cars_paper/data/subject1/h1/0/cam4/hand_pose_mano/000064.txt
/Users/dennisbaumann/cars_paper/data/subject1/h1/0/cam4/hand_pose_mano/000073.txt
/Users/dennisbaumann/cars_paper/data/subject1/h1/0/cam4/hand_pose_mano/000082.txt
/Users/dennisbaumann/cars_paper/data/subject1/h1/0/cam4/hand_pose_mano/0000

In [54]:
for n, pose in enumerate(obj_pose):
    np.save('/Users/dennisbaumann/cars_paper/data/obj_8_train/' + f'{n+1:03d}.npy', pose)
